In [15]:
# Data Manipulation
import numpy as np
import pandas as pd
import shap

# Visualizaiton
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Get Dataset from Scikit-learn
from sklearn.datasets import load_boston

# Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Regressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

# Metrics
from sklearn.metrics import mean_squared_error

In [5]:
# Load data
boston_data = load_boston()

In [16]:
# Subsume into a dataframe
boston = pd.DataFrame(boston_data['data'], columns=boston_data['feature_names'])
# Add target to the dataframe
boston['MEDV'] = boston_data.target

boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [11]:
# Define feature and target
X = boston.drop('MEDV', axis=1)      # Feature
y = boston['MEDV']                   # Label

In [12]:
# Splitting the datasets into training and testing data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Output the train and test data size
print(f"Train and Test Size {len(X_train)}, {len(X_test)}")

Train and Test Size 404, 102


In [13]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

In [14]:
# fit/train model
pipe.fit(X_train, y_train)

# predict labels
y_pred = pipe.predict(X_test)

# metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse:0.4}')
print(f'RMSE: {rmse:0.4}')
print(f'R^2 Train: {pipe.score(X_train, y_train):0.4}')
print(f'R^2 Test: {pipe.score(X_test, y_test):0.4}')

MSE: 33.45
RMSE: 5.784
R^2 Train: 0.773
R^2 Test: 0.5892


**Regularized Regression**

If the coeficients are too large in linear regression, it can lead to overfitting the model. We do regularization to over come such issues that penalises large coefficients. LASSO and Ridge regression are penalty regression that prevent over-fitting from the simple linear regression.

In [17]:
laso = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Lasso(alpha=0.1))
])

In [18]:
# fit lasso model
laso.fit(X_train, y_train)
print(f'R^2 Train: {laso.score(X_train, y_train):0.4}')
print(f'R^2 Test: {laso.score(X_test, y_test):0.4}')

R^2 Train: 0.7677
R^2 Test: 0.5664


In [19]:
laso['regressor'].coef_

array([-0.66346811,  0.70152397, -0.13072368,  0.5889339 , -1.35874893,
        2.72275426, -0.        , -2.14093195,  0.64085327, -0.65877899,
       -2.17221721,  0.60266575, -3.61579998])

In [20]:
alpha_range = np.linspace(0.01,1,50)
# alpha_range = 10**np.linspace(2,-2,100)*0.5
la_coef = []

for i in alpha_range:
    laso = Pipeline([('scaler', StandardScaler()), ('regressor', Lasso(alpha=i))])
    laso.fit(X, y)
    la_coef.append(laso['regressor'].coef_)

In [21]:
def plot_coeff(alpha_range, coef, modelname):
    
    # Plot Coefficients
    fig = plt.figure(figsize=(20,8))
    ax = plt.axes()
    
    ax.plot(alpha_range, coef)
    ax.set_xscale('log')
    ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
    ax.set_title(f'{modelname} coefficients as a function of the regularization')
    ax.set_xlabel('$\lambda$')
    ax.set_ylabel('$\mathbf{w}$')    

In [22]:
# Plot Coefficients
plot_coeff(alpha_range, la_coef, 'Lasso')

**Ridge**

In ridge regression, the cost function is altered by adding a L2 penality equivalent to square of the magnitude of the coefficients. The Ridge regression shrinks the coefficients and helps to reduce the multi-collinearity. As seen from the above graph, when λ → 0, the cost function becomes similar to the linear regression cost function.

In [23]:
rid = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge(alpha=1))
])

In [24]:
# fit ridge model
rid.fit(X, y)
print(f'R^2 Train: {rid.score(X_train, y_train):0.4}')
print(f'R^2 Test: {rid.score(X_test, y_test):0.4}')

R^2 Train: 0.77
R^2 Test: 0.6184


In [25]:
rid['regressor'].coef_

array([-0.91987132,  1.06646104,  0.11738487,  0.68512693, -2.02901013,
        2.68275376,  0.01315848, -3.07733968,  2.59153764, -2.0105579 ,
       -2.05238455,  0.84884839, -3.73066646])

In [26]:
# alpha_range = np.arange(1,2000,100)
alpha_range = 10**np.linspace(6,-2,100)*0.5
rid_coef = []

for i in alpha_range:
    rid = Pipeline([('scaler', StandardScaler()), ('regressor', Ridge(alpha=i))])
    rid.fit(X, y)
    rid_coef.append(rid['regressor'].coef_)

In [31]:
# Plot Coefficients
plot_coeff(alpha_range, rid_coef, 'Ridge')

**Features Selection**

Method 1: VIF

1. VIF starts at 1 and has no upper limit
2. VIF = 1, no correlation between the independent variable and the other variables
3. VIF exceeding 5 or 10 indicates high multicollinearity between this independent variable and the others


In [33]:
# Import VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [34]:
# For each X, calculate VIF and save in dataframe
def vif(X):
    
    # perform feature scaling
    scaler = StandardScaler()
    xs = scaler.fit_transform(X)
    
    # subsume into a dataframe
    vif = pd.DataFrame()
    vif["Features"] = X.columns
    vif["VIF Factor"] = [variance_inflation_factor(xs, i) for i in range(xs.shape[1])]
    
    return vif

In [35]:
# List scores
vif(X).round(2)

,Features,VIF Factor
0,CRIM,1.79
1,ZN,2.30
2,INDUS,3.99
3,CHAS,1.07
4,NOX,4.39
5,RM,1.93
6,AGE,3.10
7,DIS,3.96
8,RAD,7.48
9,TAX,9.01


In [36]:
# Drop VIF score > 5
newX = X.drop(['TAX', 'RAD'],axis=1)

In [37]:
# Scores in ascending values
vif(newX).sort_values(by="VIF Factor")

,Features,VIF Factor
3,CHAS,1.057805
9,B,1.316559
0,CRIM,1.478206
8,PTRATIO,1.496077
5,RM,1.872532
1,ZN,2.154483
10,LSTAT,2.936487
6,AGE,3.075755
2,INDUS,3.179166
4,NOX,3.901348


In [38]:
# Filter first six features
X_method1 = X[['CHAS', 'B', 'CRIM', 'PTRATIO', 'RM', 'ZN']]

In [39]:
# fit/train model
pipe.fit(X_method1, y)

# predict labels
y_pred = pipe.predict(X_method1)

print(y_pred[:10])
print(f'R^2: {pipe.score(X_method1, y):0.4}')

[28.57799551 24.78825544 30.16715122 28.03678666 29.13085894 23.98038835
 24.50415657 25.65810523 21.62521274 24.29070206]
R^2: 0.6273


**Method 2: SelectKBest**

In [40]:
# Feature Selection
from sklearn.feature_selection import f_regression, SelectKBest, SelectPercentile

In [41]:
# SelectKBest
# selector1 = SelectPercentile(f_regression, percentile=25)
method2 =  SelectKBest(f_regression, k=6) 

# Fit the model
method2.fit(X,y)

SelectKBest(k=6, score_func=<function f_regression at 0x0000024F065E14C0>)

In [42]:
# Show selected features
method2.get_support(indices=True)

array([ 2,  4,  5,  9, 10, 12], dtype=int64)

In [43]:
# Iterate the score
for f, s in zip(X.columns, method2.scores_):
    print(f'F-score: {s:0.4} for feature {f}')

F-score: 89.49 for feature CRIM
F-score: 75.26 for feature ZN
F-score: 154.0 for feature INDUS
F-score: 15.97 for feature CHAS
F-score: 112.6 for feature NOX
F-score: 471.8 for feature RM
F-score: 83.48 for feature AGE
F-score: 33.58 for feature DIS
F-score: 85.91 for feature RAD
F-score: 141.8 for feature TAX
F-score: 175.1 for feature PTRATIO
F-score: 63.05 for feature B
F-score: 601.6 for feature LSTAT


In [44]:
# Filter six features with highest score
X_method2 = X[['INDUS', 'NOX', 'RM', 'TAX', 'PTRATIO', 'LSTAT']]

In [45]:
# fit/train model
pipe.fit(X_method2, y)

# predict labels
y_pred = pipe.predict(X_method2)

print(y_pred[:10])
print(f'R^2: {pipe.score(X_method2, y):0.4}')

[30.57014999 26.10398705 32.45084162 31.0219246  30.41087418 27.1380861
 24.46192059 21.5381886  13.13415681 21.87515864]
R^2: 0.681


In [46]:
# check the coefficients
pipe['regressor'].coef_

array([ 0.59754458, -0.39395547,  3.26810054, -0.48846049, -1.9764135 ,
       -3.89469824])

In [47]:
# Feature Selection using RFE
from sklearn.feature_selection import RFECV, RFE

In [48]:
# Method 3
method3 = RFE(LinearRegression(), n_features_to_select=6, step=1)
method3.fit(X,y)

RFE(estimator=LinearRegression(), n_features_to_select=6)

In [49]:
RFE(estimator=LinearRegression(), n_features_to_select=6)

RFE(estimator=LinearRegression(), n_features_to_select=6)

In [50]:
# Check the selected position
method3.support_

array([False, False, False,  True,  True,  True, False,  True, False,
       False,  True, False,  True])

In [51]:
# Get the feature ranking
method3.ranking_

array([3, 5, 4, 1, 1, 1, 8, 1, 2, 6, 1, 7, 1])

In [52]:
http://localhost:8888/notebooks/Downloads/LinerRegression.ipynb## Select Six Features
min_value = min(method3.ranking_)
col = [i for i, x in enumerate(method3.ranking_) if x == min_value]
col

[3, 4, 5, 7, 10, 12]

In [66]:
# Iterate to get features
for i in range(len(col)):
    print(X.columns[col[i]])

CHAS
NOX
RM
DIS
PTRATIO
LSTAT


In [54]:
# Filter selected features
X_method3 = X[['CHAS', 'NOX', 'RM', 'DIS', 'PTRATIO', 'LSTAT']]

In [55]:
# fit/train model
pipe.fit(X_method3, y)

# predict labels
y_pred = pipe.predict(X_method3)

print(y_pred[:10])
print(f'R^2: {pipe.score(X_method3, y):0.4}')

[31.0142456  25.79278275 31.84611084 29.74856961 28.99930186 26.11951419
 23.13348366 19.51563196 10.99759368 19.25866209]
R^2: 0.7158


In [56]:
# check the coefficients
pipe['regressor'].coef_

array([ 0.82321941, -2.16945087,  2.88617319, -2.40778286, -2.16874483,
       -4.06526959])

    **RFECV**

In [57]:
# Method 4
method4 = RFECV(LinearRegression(), cv=10)
method4.fit(X,y)

RFECV(cv=10, estimator=LinearRegression())

In [58]:
# Get the selected features with CV
method4.n_features_

6

In [59]:
# Get the index of the selected features
method4.get_support(indices=True)

array([ 3,  4,  5,  7, 10, 12], dtype=int64)

In [60]:
# Check the selected position
method4.support_

array([False, False, False,  True,  True,  True, False,  True, False,
       False,  True, False,  True])

In [61]:
# Get the feature ranking
method4.ranking_

array([3, 5, 4, 1, 1, 1, 8, 1, 2, 6, 1, 7, 1])

In [63]:
# Select Six Features
min_value = min(method4.ranking_)
col = [i for i, x in enumerate(method4.ranking_) if x == min_value]
col

[3, 4, 5, 7, 10, 12]

In [65]:
# Iterate to get features
for i in range(len(col)):
    print(X.columns[col[i]])

CHAS
NOX
RM
DIS
PTRATIO
LSTAT


In [67]:
# Filter selected features
X_method4 = X[['CHAS', 'NOX', 'RM', 'DIS', 'PTRATIO', 'LSTAT']]

In [68]:
# fit/train model
pipe.fit(X_method4, y)

# predict labels
y_pred = pipe.predict(X_method4)

print(y_pred[:10])
print(f'R^2: {pipe.score(X_method4, y):0.4}')

[31.0142456  25.79278275 31.84611084 29.74856961 28.99930186 26.11951419
 23.13348366 19.51563196 10.99759368 19.25866209]
R^2: 0.7158


In [69]:
# check the coefficients
pipe['regressor'].coef_

array([ 0.82321941, -2.16945087,  2.88617319, -2.40778286, -2.16874483,
       -4.06526959])

**Method 5: SHAP**

In [71]:
# 100 instances for use as the background distribution
X100 = shap.utils.sample(X, 100, random_state=42)
pipe.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', LinearRegression())])

In [72]:
# compute the SHAP values for the linear model
explainer = shap.Explainer(pipe.predict, X100)
shap_values = explainer(X)

Permutation explainer: 507it [00:39, 10.87it/s]                                                                        


In [73]:
shap.plots.beeswarm(shap_values, max_display=14)

In [74]:
# Filter selected features
X_method5 = X[['LSTAT', 'DIS', 'RM', 'PTRATIO', 'CRIM']]

In [75]:
# fit/train model
pipe.fit(X_method5, y)

# predict labels

y_pred = pipe.predict(X_method5)

print(y_pred[:10])
print(f'R^2: {pipe.score(X_method5, y):0.4}')

[31.63761273 25.5236182  32.0243847  30.38600726 29.5358719  26.50795486
 23.71933951 19.96956586 10.8068779  20.1215183 ]
R^2: 0.6958
